In [1]:
import requests
import json
import pyspark as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import scipy.stats as stats

pd.options.display.max_columns = None
pd.options.display.max_rows = None

df=pd.read_csv('FINAL.csv')
df=df.drop(['Unnamed: 0'],axis=1)

In [2]:
planeswalker_df=df[df.types=='Planeswalker']
planeswalker_df=planeswalker_df.sort_values('name')
planes_foil=planeswalker_df[planeswalker_df.hasFoil==1]

legendary_df=df[(df.supertypes=='Legendary')&(df.types=='Creature')]
legendary_df=legendary_df[legendary_df.hasFoil==1]
legendary_df=legendary_df.sort_values('name')

legendary_df.reset_index(inplace=True)
planes_foil.reset_index(inplace=True)

other_df=df.query('types!="Creature" and supertypes!="Legendary"')
other_df=other_df.query('hasFoil==1')
other_df.query('supertypes!="Planeswalker"',inplace=True)

other_foil=other_df[['name','setCode','rarity','types','TCGPlayer_Foil_Price','TCGPlayer_Foil_Change','TCGPlayer_Foil_BLPrice','TCGPlayer_Foil_Profit','edhrecRank']]
other_normal=other_df[['name','setCode','rarity','types','TCGPlayer_Norm_Price','TCGPlayer_Norm_Change','TCGPlayer_Norm_BLPrice','TCGPlayer_Norm_Profit','edhrecRank']]

df_new=pd.concat([legendary_df,planes_foil])
foils=df_new[['name','setCode','rarity','types','TCGPlayer_Foil_Price','TCGPlayer_Foil_Change','TCGPlayer_Foil_BLPrice','TCGPlayer_Foil_Profit','edhrecRank']]
normal=df_new[['name','setCode','rarity','types','TCGPlayer_Norm_Price','TCGPlayer_Norm_Change','TCGPlayer_Norm_BLPrice','TCGPlayer_Norm_Profit','edhrecRank']]

In [14]:
other_foil.dropna(inplace=True)
other_normal.dropna(inplace=True)
normal.dropna(inplace=True)
foils.dropna(inplace=True)

/home/chris/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
# calculating the standard deviation and t-test of the samples
ofmean=np.mean(other_foil.TCGPlayer_Foil_Profit)
cfmean=np.mean(foils.TCGPlayer_Foil_Profit)

foiln=other_foil.TCGPlayer_Foil_Profit
foilc=foils.TCGPlayer_Foil_Profit

stdn=np.std(foiln)
stdc=np.std(foilc)

stats.ttest_ind_from_stats(mean1=ofmean,std1=stdn,nobs1=len(other_foil),
                          mean2=cfmean,std2=stdc,nobs2=len(foils))

Ttest_indResult(statistic=-8.792216310822186, pvalue=1.648166059851758e-18)

In [26]:
onmean=np.mean(other_normal.TCGPlayer_Norm_Profit)
cnmean=np.mean(normal.TCGPlayer_Norm_Profit)

normn=other_normal.TCGPlayer_Norm_Profit
normc=normal.TCGPlayer_Norm_Profit

stdn=np.std(normn)
stdc=np.std(normc)

stats.ttest_ind_from_stats(mean1=onmean,std1=stdn,nobs1=len(other_normal),
                          mean2=cnmean,std2=stdc,nobs2=len(normal))

Ttest_indResult(statistic=-16.088787185488087, pvalue=1.3572386834784576e-57)

In [27]:
ofmean=np.mean(other_foil.TCGPlayer_Foil_Change)
cfmean=np.mean(foils.TCGPlayer_Foil_Change)

foiln=other_foil.TCGPlayer_Foil_Change
foilc=foils.TCGPlayer_Foil_Change

stdn=np.std(foiln)
stdc=np.std(foilc)

stats.ttest_ind_from_stats(mean1=ofmean,std1=stdn,nobs1=len(other_foil),
                          mean2=cfmean,std2=stdc,nobs2=len(foils))

Ttest_indResult(statistic=3.6403556285371783, pvalue=0.00027331322437496437)

In [28]:
onmean=np.mean(other_normal.TCGPlayer_Norm_Change)
cnmean=np.mean(normal.TCGPlayer_Norm_Change)

normn=other_normal.TCGPlayer_Norm_Change
normc=normal.TCGPlayer_Norm_Change

stdn=np.std(normn)
stdc=np.std(normc)

stats.ttest_ind_from_stats(mean1=cnmean,std1=stdc,nobs1=len(normal),
                          mean2=onmean,std2=stdn,nobs2=len(other_normal))

Ttest_indResult(statistic=-2.204281063717118, pvalue=0.027524930076921545)

In [ ]:
pd.plotting.scatter_matrix(normal, figsize=(10, 10), s=100);

In [ ]:
pd.plotting.scatter_matrix(foils, figsize=(10, 10), s=100);

In [ ]:
foils.info()

In [ ]:
normal.info()

In [ ]:
foils.plot.scatter(x='TCGPlayer_Foil_Change',

                      y='edhrecRank',

                      c='DarkBlue')

In [ ]:
normal.plot.scatter(x='TCGPlayer_Norm_Change',

                      y='edhrecRank',

                      c='DarkBlue')

In [ ]:
# price change vs price
# looks like the higher the price of the card the less volatility in the price
foils.plot.scatter(x='TCGPlayer_Foil_Price',

                      y='TCGPlayer_Foil_Change',

                      c='DarkBlue')

In [ ]:
# price change vs 
normal.plot.scatter(x='TCGPlayer_Norm_Price',

                      y='TCGPlayer_Norm_Change',

                      c='DarkBlue')

In [ ]:
#Price of normal cards in the sample vs the profit that could be made
# for the most part it is what I expected to see. The higher the card price the greater potential for profit

normal.plot.scatter(x='TCGPlayer_Norm_Price',

                      y='TCGPlayer_Norm_Profit',

                      c='DarkBlue')

In [ ]:
# Price of a normal card in the sample vs the profit that could be made
# for the most part it is what I expected to see. The higher the card price the greater potential for profit.

foils.plot.scatter(x='TCGPlayer_Foil_Price',

                      y='TCGPlayer_Foil_Profit',

                      c='DarkBlue')

Interesting that there are negative profit numbers for normal cards and foil cards in the data sample. This probably indicates a supply problem. Some stores are selling cards for less than other stores are buying cards for. Maybe they are buying cards to hold, predicting the price of this card going up in the future?

In [ ]:
foils.plot.scatter(x='TCGPlayer_Foil_Profit',

                      y='TCGPlayer_Foil_Change',

                      c='DarkBlue')

In [ ]:
normal.plot.scatter(x='TCGPlayer_Norm_Profit',

                      y='TCGPlayer_Norm_Change')

In [ ]:
def emperical_distribution(x, data):
    weight = 1.0 / len(data)
    count = np.zeros(shape=len(x))
    for datum in data:
        count = count + np.array(x >= datum)
    return weight * count

def one_dim_scatterplot(data, ax, jitter=0.2, **options):
    ## why jitter? especially for bootstraping later
    if jitter:
        jitter = np.random.uniform(-jitter, jitter, size=data.shape)
    else:
        jitter = np.repeat(0.0, len(data))
    ax.scatter(data, jitter, **options)
    mean = np.mean(data)
    ax.scatter([mean], 0, c="black", s=80)
    ax.yaxis.set_ticklabels([])
    ax.set_ylim([-1, 1])
    ax.tick_params(axis='both', which='major', labelsize=15)

In [ ]:
data=foils.TCGPlayer_Foil_Change
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

x = np.linspace(-3, 3, num=250)
ax[0].plot(x, emperical_distribution(x, data), linewidth=2)
ax[0].set_ylim(-0.05, 1.05)
ax[0].set_xlim(-2, 2)
# ax[0].tick_params(axis='both', which='major', labelsize=15)
one_dim_scatterplot(data, ax[1], s=25)
_ = ax[0].set_title("The Empirical Distribution Function for Foil Price Change")
print(f'The mean of the data is:{np.mean(data)}')

In [ ]:
data=normal.TCGPlayer_Norm_Change
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

x = np.linspace(-3, 3, num=250)
ax[0].plot(x, emperical_distribution(x, data), linewidth=2)
ax[0].set_ylim(-0.05, 1.05)
ax[0].set_xlim(-2, 2)
# ax[0].tick_params(axis='both', which='major', labelsize=15)
one_dim_scatterplot(data, ax[1], s=25)
_ = ax[0].set_title("The Empirical Distribution Function for Normal Card Price Change")
print(f'The mean of the data is:{np.mean(data)}')